# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import DistilBertModel, AdamW, AutoModel, AlbertModel
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.metrics import classification_report

c:\Users\anuja\anaconda3\envs\gpu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataloaders

In [2]:
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        input_ids = self.inputs['input_ids'][idx]
        attention_mask = self.inputs['attention_mask'][idx]
        label = self.labels[idx]
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

# Creating Model Definition

In [26]:
class CNNLSTMClassifier(nn.Module):
    def __init__(self, distilbert, cnn_out_channels=64, lstm_hidden_dim=64, num_classes=2):
        super(CNNLSTMClassifier, self).__init__()
        self.distilbert = distilbert
        self.cnn = nn.Conv1d(in_channels=768, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(cnn_out_channels, lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, num_classes)
        
    def forward(self, input_ids, attention_mask):
        with torch.set_grad_enabled(self.distilbert.training):
            distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = distilbert_output.last_hidden_state.permute(0, 2, 1)  # (batch, embed_dim, seq_len)
        
        cnn_out = self.cnn(embeddings)
        
        lstm_out, _ = self.lstm(cnn_out.permute(0, 2, 1))  # (batch, seq_len, lstm_hidden_dim)
        
        logits = self.fc(lstm_out[:, -1, :])  # Use last hidden state for classification
        return logits

In [3]:
class CNNLSTMClassifier(nn.Module):
    def __init__(self, bert, in_channels, cnn_out_channels=64, lstm_hidden_dim=64, num_classes=2):
        super(CNNLSTMClassifier, self).__init__()
        self.bert = bert
        self.cnn = nn.Conv1d(in_channels=in_channels, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(cnn_out_channels, lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, num_classes)
        
    def forward(self, input_ids, attention_mask):
        with torch.set_grad_enabled(self.bert.training):
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = bert_output.last_hidden_state.permute(0, 2, 1)  # (batch, embed_dim, seq_len)
        
        cnn_out = self.cnn(embeddings)
        
        lstm_out, _ = self.lstm(cnn_out.permute(0, 2, 1))  # (batch, seq_len, lstm_hidden_dim)
        
        logits = self.fc(lstm_out[:, -1, :])  # Use last hidden state for classification
        return logits

# Training and Evaluation

In [4]:
train_dataset = pd.read_csv('cleaned_dataset.csv')
dev_dataset = pd.read_csv('cleaned_dev_dataset.csv')

In [5]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=1, accumulation_steps=10, device='cuda'):
    for epoch in range(num_epochs):
        total_loss = 0
        for i, batches in enumerate(tqdm(dataloader)):
            input_ids = batches['input_ids'].to(device)
            attention_mask = batches['attention_mask'].to(device)
            labels = batches['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

    return model

In [6]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    print("Classification Report:\n")
    with torch.no_grad():
        for batches in tqdm(dataloader):
            input_ids = batches['input_ids'].to(device)
            attention_mask = batches['attention_mask'].to(device)
            labels = batches['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    report = classification_report(true_labels, predictions, digits=4)
    print(report)

In [7]:
def test_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    print("Classification Report:\n")
    with torch.no_grad():
        for batches in tqdm(dataloader):
            input_ids = batches['input_ids'].to(device)
            attention_mask = batches['attention_mask'].to(device)
            labels = batches['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    report = classification_report(true_labels, predictions, digits=4)
    print(report)
    return predictions, true_labels

# DistilBert

In [34]:
train_DB_inputs = torch.load('intermediates/DB_inputs.pt')
dev_DB_inputs = torch.load('intermediates/DB_dev_inputs.pt')
test_DB_inputs = torch.load('intermediates/DB_test_inputs.pt')

train_labels = torch.load('intermediates/labels.pt')
dev_labels = torch.load('intermediates/labels_dev.pt')
test_labels = torch.load('intermediates/labels_test.pt')

C:\Users\anuja\AppData\Local\Temp\ipykernel_27884\3470213625.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_DB_inputs = torch.load('intermediates/DB_inputs.pt')
C

In [35]:
train_custom_dataset = CustomDataset(inputs=train_DB_inputs, labels=train_labels)
dev_custom_dataset = CustomDataset(inputs=dev_DB_inputs, labels=dev_labels)
test_custom_dataset = CustomDataset(inputs=test_DB_inputs, labels=test_labels)

train_dataloader = DataLoader(train_custom_dataset, batch_size=10, shuffle=True)
dev_dataloader = DataLoader(dev_custom_dataset, batch_size=10)
test_dataloader = DataLoader(test_custom_dataset, batch_size=10)

In [25]:
distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
DB_model = CNNLSTMClassifier(distilbert, in_channels=768)
DB_model.train()

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(DB_model.parameters(), lr=2e-5) #learing rate used by baseline from COLING 2025

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
DB_model.to(device)

In [ ]:
DB_model = train_model(DB_model, train_dataloader, criterion, optimizer, num_epochs=1, accumulation_steps=10, device=device)

  0%|          | 0/61077 [00:00<?, ?it/s]c:\Users\Vishal Perumal\anaconda3\envs\gpu\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 61077/61077 [5:02:40<00:00,  3.36it/s]  

Epoch 1/1, Loss: 0.2886


In [ ]:
torch.save(DB_model, 'models/DB_model.pt')

In [30]:
DB_model = torch.load('models/DB_model.pt')

C:\Users\anuja\AppData\Local\Temp\ipykernel_27884\4132632868.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DB_model = torch.load('models/DB_model.pt')


In [33]:
evaluate_model(DB_model, dev_dataloader, device)

Classification Report:



  0%|          | 0/26176 [00:00<?, ?it/s]

100%|██████████| 26176/26176 [1:22:24<00:00,  5.29it/s]


              precision    recall  f1-score   support

           0     0.9033    0.8553    0.8786     98328
           1     0.9157    0.9449    0.9300    163430

    accuracy                         0.9113    261758
   macro avg     0.9095    0.9001    0.9043    261758
weighted avg     0.9110    0.9113    0.9107    261758



In [38]:
DB_preds, DB_true_labels = test_model(DB_model, test_dataloader, device)

Classification Report:



100%|██████████| 7395/7395 [23:32<00:00,  5.24it/s]


              precision    recall  f1-score   support

           0     0.8106    0.6062    0.6937     34675
           1     0.7156    0.8749    0.7873     39266

    accuracy                         0.7489     73941
   macro avg     0.7631    0.7406    0.7405     73941
weighted avg     0.7601    0.7489    0.7434     73941



# TinyBert

In [ ]:
train_TB_inputs = torch.load('intermediates/TB_inputs.pt')
dev_TB_inputs = torch.load('intermediates/TB_dev_inputs.pt')

train_labels = torch.load('intermediates/labels.pt')
dev_labels = torch.load('intermediates/labels_dev.pt')

In [ ]:
train_custom_dataset = CustomDataset(inputs=train_TB_inputs, labels=train_labels)
dev_custom_dataset = CustomDataset(inputs=dev_TB_inputs, labels=dev_labels)

train_dataloader = DataLoader(train_custom_dataset, batch_size=10, shuffle=True)
dev_dataloader = DataLoader(dev_custom_dataset, batch_size=10)

In [ ]:
tinybert = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

In [ ]:
TB_model = CNNLSTMClassifier(tinybert, in_channels=312)
TB_model.train()

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(TB_model.parameters(), lr=2e-5) #learing rate used by baseline from COLING 2025

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
TB_model.to(device)

In [ ]:
TB_model = train_model(TB_model, train_dataloader, criterion, optimizer, num_epochs=1, accumulation_steps=10, device=device)

100%|██████████| 61077/61077 [2:51:55<00:00,  5.92it/s]  

Epoch 1/1, Loss: 0.3766


In [ ]:
torch.save(TB_model, 'models/TB_model.pt')

In [ ]:
evaluate_model(TB_model, dev_dataloader, device)

100%|██████████| 26176/26176 [26:23<00:00, 16.53it/s]


Classification Report:
               precision    recall  f1-score   support

           0     0.8366    0.8328    0.8347     98328
           1     0.8997    0.9021    0.9009    163430

    accuracy                         0.8761    261758
   macro avg     0.8681    0.8674    0.8678    261758
weighted avg     0.8760    0.8761    0.8760    261758



# ALBERT

In [8]:
train_AB_inputs = torch.load('intermediates/AB_inputs.pt')
dev_AB_inputs = torch.load('intermediates/AB_dev_inputs.pt')
test_AB_inputs = torch.load('intermediates/AB_test_inputs.pt')

train_labels = torch.load('intermediates/labels.pt')
dev_labels = torch.load('intermediates/labels_dev.pt')
test_labels = torch.load('intermediates/labels_test.pt')

C:\Users\anuja\AppData\Local\Temp\ipykernel_30120\2017279522.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_AB_inputs = torch.load('intermediates/AB_inputs.pt')
C

In [9]:
train_custom_dataset = CustomDataset(inputs=train_AB_inputs, labels=train_labels)
dev_custom_dataset = CustomDataset(inputs=dev_AB_inputs, labels=dev_labels)
test_custom_dataset = CustomDataset(inputs=test_AB_inputs, labels=test_labels)

train_dataloader = DataLoader(train_custom_dataset, batch_size=10, shuffle=True)
dev_dataloader = DataLoader(dev_custom_dataset, batch_size=10)
test_dataloader = DataLoader(test_custom_dataset, batch_size=10)

In [13]:
albert = AlbertModel.from_pretrained("albert-small")

OSError: albert-small is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [11]:
AB_model = CNNLSTMClassifier(albert, in_channels=768)
AB_model.train()

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(AB_model.parameters(), lr=2e-5) #learing rate used by baseline from COLING 2025

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
AB_model.to(device)

c:\Users\anuja\anaconda3\envs\gpu_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using device: cuda


CNNLSTMClassifier(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)


In [12]:
AB_model = train_model(AB_model, train_dataloader, criterion, optimizer, num_epochs=1, accumulation_steps=10, device=device)

  0%|          | 287/61077 [58:37<206:58:36, 12.26s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(AB_model, 'models/AB_model.pt')

In [ ]:
evaluate_model(AB_model, dev_dataloader, device)

In [ ]:
AB_preds, AB_true_labels = test_model(AB_model, test_dataloader, device)